# DataStore API

CTI Python STIX2 features a new interface for pulling and pushing STIX2 content. The new interface consists of DataStore, DataSource and DataSink constructs: a DataSource for pulling STIX2 content, a DataSink for pushing STIX2 content, and a DataStore for pulling/pushing.

### DataSource API (snapshot)
* **get()**          - search/retrieve most current STIX SDO/SRO via its ID
* **all_versions()** - search/retrieve all versions of STIX SDO/SRO via its id
* **query()**       - search/retrieve STIX SDO/SRO(s) via search filters
* **add_filters()**  - attach filter(s) to the DataSource
 
### DataSink API (snapshot)
* **add()** - add a set of STIX SDO/SRO to a target endpoint.
 
### DataStore API (snapshot)  

(super set of the DataSource and DataSink)
* **get()**
* **all_versions()**
* **query()**
* **add_filters()**
* **add()**

The DataStore, DataSource, DataSink (referred to as "DataStore suite") APIs are not referenced directly by a user but are used as base classes, which are then sublcassed into real DataStore suite(s). CTI Python STIX2 provides for the DataStore suites of **FileSystem**, **Memory**, and **TAXII**. Users are also encrouraged subclassing the base Data suite and creating their own custom DataStore suites.

## CompositeDataSource

**CompositeDataSource** is an available controller that can be used as a single interface to a set of defined DataSources. The purpose of this controller is allow for the grouping of **DataSources** and making get/query calls to a set of DataSources in one API call. **CompositeDataSource** can be used to organize/group **DataSources**, federate get()/all_versions()/query() calls, and reduce user code.

**CompositeDataSource** is just a wrapper around a set of defined **DataSources** (e.g. FileSystemSource) that federates get()/all_versions()/query() calls individually to each of the attached **DataSources** , collects the results from each **DataSource** and returns them.

Filters can be attached to **CompositeDataSources** just as they can be done to **DataStores** and **DataSources**. When get()/all_versions()/query() calls are made to the **CompositeDataSource**, it will pass along any query filters from the call and any of its own filters to the attached **DataSources**. To which, those attached **DataSources** may have their own attached filters as well. The effect is that all the filters are eventually combined when the get()/all_versions()/query() call is actually executed within a **DataSource**. 

A **CompositeDataSource** can also be attached to a **CompositeDataSource** for multiple layers of grouped **DataSources**.


### CompositeDataSource API (snapshot)

* **get()**          - search/retrieve from CompositeDataSource most current STIX object via its ID
* **all_versions()** - search/retrieve from CompositeDataSource all versions of STIX object via its ID
* **query()**        - search/retrieve from CompositeDataSource STIX object(s) via search filters
* **add_data_source()** - Attach a DataSource to the CompositeDataSource
* **remove_data_source()** - Remove DataSource from the CompositeDataSource
* **add_filters()**  - attach filter(s) to the CompositeDataSource

### CompositeDataSource Examples


In [1]:
import sys
sys.path.append("/home/michael/oasis-python-stix2/cti-python-stix2/")
sys.path

from stix2 import CompositeDataSource, FileSystemStore


# TO BE COMPLETED, WAITING ON TAXIICollection,
# want to put a TAXIICollection in here

## Filters

The CTI Python STIX2 **DataStore** suites, to include **FileSystem**, **Memory** and **TAXII** all use the **Filters** module to allow for the querying of STIX content. The basic functionality is that filters can be created and supplied everytime to calls to **query()**, and/or attached to a **DataStore** so that every future query placed to that **DataStore** is evaluated against the attached filters, supplemented with any further filters supplied with the query call. Attached filters can also be removed from **DataStores**.

Filters are very simple, as they consist of a STIX object common property name, comparison operator and an object property value (i.e. value to compare to). Currently, CTI Python STIX2 supports **ONLY** STIX 2 object common properties:

* created
* created_by_ref
* external_references.source_name
* external_references.description
* external_references.url
* external_references.external_id
* granular_markings.marking_ref
* granular_markings.selectors
* id
* labels
* modified
* object_marking_refs
* revoked
* type

Supported operators on these properties:

* =
* !=
* in
* >
* < 
* ```>=```
* <=

To align with STIX 2 object common properties, the value types of the common property values must be one of these (python) types:

* bool
* dict
* float
* int
* list
* str
* tuple

### Filter Examples

In [2]:
import sys
sys.path.append("/home/michael/oasis-python-stix2/cti-python-stix2/")
sys.path
from stix2 import Filter

# create filter for STIX objects that have external references to MITRE ATT&CK framework
f = Filter("external_references.source_name", "=", "mitre-attack")

# create filter for STIX objects that are not of SDO type Attack-Pattnern
f1 = Filter("type", "!=", "attack-pattern")

# create filter for STIX objects that have the "threat-report" label
f2 = Filter("labels", "in", "threat-report")

# create filter for STIX objects that have been modified past the timestamp
f3 = Filter("modified", ">=", "2017-01-28T21:33:10.772474Z")

# create filter for STIX objects that have been revoked
f4 = Filter("revoked", "=", True)

For Filters to be applied to a query, they must be either supplied with the query call or attached a **DataStore**, more specifically to **DataSource** whether that **DataSource** is a part of a **DataStore** or stands by itself. 

In [3]:
from stix2 import MemoryStore, FileSystemStore, FileSystemSource

fs = FileSystemStore("/home/michael/Desktop/sample_stix2_data")
fs_source = FileSystemSource("/home/michael/Desktop/sample_stix2_data")

# attach filter to FileSystemStore
fs.add_filters(f)

# attach multiple filters to FileSystemStore
fs.add_filters([f1,f2])

# can also attach filters to a Source
# attach multiple filters to FileSystemSource
fs_source.add_filters([f3, f4])


mem = MemoryStore()

# As it is impractical to only use MemorySink or MemorySource,
# attach a filter to a MemoryStore
mem.add_filters(f)

# attach multiple filters to a MemoryStore
mem.add_filters([f1,f2])